In [58]:
from transformers.data.data_collator import default_data_collator
import torch.utils.data as Data
from datasets import load_dataset
from transformers import AutoTokenizer
import random

In [59]:
train_dataset = load_dataset(path='dair-ai/emotion')['train']
train_dataset[:10]

Using the latest cached version of the dataset since dair-ai/emotion couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'split' at C:\Users\duanm\.cache\huggingface\datasets\dair-ai___emotion\split\0.0.0\cab853a1dbdf4c42c2b3ef2173804746df8825fe (last modified on Wed Jun 18 11:28:24 2025).


{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy',
  'ive been feeling a little burdened lately wasnt sure why that was',
  'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny',
  'i feel as confused about life as a teenager or as jaded as a year old man',
  'i have been with petronas for years i feel that petronas has performed well and made a huge profit',
  'i feel romantic too'],
 'label': [0, 0, 3, 2, 3, 0, 5, 4, 1, 2]}

In [60]:
batch_size = 4

In [61]:
tokenizer = AutoTokenizer.from_pretrained('E:\huggingface_models\Qwen3-0.6B')
train_dataset = train_dataset.map(lambda samples: tokenizer(samples["text"], padding=True), batched=True, batch_size=batch_size)
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 16000
})

In [62]:
for i in train_dataset.select(range(0, 10)):
    print(len(i['input_ids']))

21
21
21
21
24
24
24
24
23
23


In [63]:
add_data = [str(i) for i in range(16000)]
train_dataset = train_dataset.add_column("xx", add_data)  # 非数值类型列
yy_list = []
for i in range(0, len(train_dataset)):
    if i % batch_size == 0:
        length = random.randint(32, 64)
    yy_list.append(list(range(0, length)))
train_dataset = train_dataset.add_column("yy", yy_list)  # 数值类型列
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask', 'xx', 'yy'],
    num_rows: 16000
})

In [ ]:
dac = Data.DataLoader(train_dataset, 
                      # 保留所有数值类型字段
                      # labal/labels字段(如果有)必须为数值类型
                      # 每个batch_size的数据必须可以转换为一个tensor
                      collate_fn=default_data_collator, 
                      batch_size=batch_size)

for i in dac:
    print(i.keys())
    print("labels: ", i['labels'])
    print("yy: ", i['yy'].shape)
    print("labels.shape: ", i['labels'].shape)
    print("input_ids.shape: ", i['input_ids'].shape)
    print("attention_mask.shape: ", i['attention_mask'].shape)
    break

dict_keys(['labels', 'input_ids', 'attention_mask', 'yy'])
labels:  tensor([0, 0, 3, 2])
yy:  torch.Size([4, 57])
labels.shape:  torch.Size([4])
input_ids.shape:  torch.Size([4, 21])
attention_mask.shape:  torch.Size([4, 21])
